## APTOS Blindness Detection EfficientNet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import os, cv2, json
from PIL import Image
from random import randint
import time

notebookstart = time.time()

## Work directory

In [ ]:
# File Parameters
WORK_DIR = "../input/aptos2019-blindness-detection/"
label_col = "diagnosis"
img_col = "id_code"
train_folder = "train_images"
test_folder = "test_images"
SUFFIX = '.png'

BATCH_SIZE = 8
EPOCHS = 10
TARGET_SIZE = 512

os.listdir(WORK_DIR)

# First look at the data

In [ ]:
print('Train images: %d' %len(os.listdir(
    os.path.join(WORK_DIR, train_folder))))

In [ ]:
train_labels = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))
label_names = train_labels[label_col].value_counts().index
label_map = {name:i for (i,name) in enumerate(label_names)}
inv_label_map = {v: k for k, v in label_map.items()}

train_labels['label_name'] = train_labels[label_col].copy()
train_labels[label_col] = train_labels[label_col].map(label_map)

train_labels[img_col] = train_labels[img_col] + SUFFIX

display(train_labels.head())

ss = pd.read_csv(os.path.join(WORK_DIR, "sample_submission.csv"))
testdex = ss[img_col].copy()
ss[img_col] = ss[img_col] + SUFFIX
display(ss.head())

In [ ]:
# Main parameters
STEPS_PER_EPOCH = len(train_labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2 / BATCH_SIZE

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize = (6, 4))

for i in ['top', 'right', 'left']:
    ax.spines[i].set_visible(False)
ax.spines['bottom'].set_color('black')

sns.countplot(y = train_labels["label_name"], order=train_labels["label_name"].value_counts().index[:5], edgecolor = 'black',
              palette = reversed(sns.color_palette("viridis", 5)))
plt.xlabel('Classes', fontfamily = 'serif', size = 15)
plt.ylabel('Count', fontfamily = 'serif', size = 15)
plt.xticks(fontfamily = 'serif', size = 12)
plt.yticks(fontfamily = 'serif', size = 12)
ax.grid(axis = 'y', linestyle = '--', alpha = 0.9)
plt.show()

In [ ]:
def img_plot(df, label=None):
    if label != None:
        sample = df[df["label_name"] == label].sample(3)
        folder = train_folder
    else:
        sample = df.sample(3)
        sample["label_name"] = "test"
        folder = test_folder
    plt.figure(figsize=(15, 5))
    for ind, (image_id, label) in enumerate(zip(sample[img_col], sample["label_name"])):
        plt.subplot(1, 3, ind + 1)
        img = cv2.imread(os.path.join(WORK_DIR, folder, image_id))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        plt.title("Label: {}\nShape: {}".format(label, img.shape))
        plt.imshow(img)
        plt.axis("off")

    plt.show()

In [ ]:
for lbl in label_names[:3]:
    img_plot(train_labels, lbl)

# Preparation for modeling

In [ ]:
train_datagen = ImageDataGenerator(validation_split = 0.2,
                                     preprocessing_function = None,
                                     rotation_range = 45,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1)

train_generator = train_datagen.flow_from_dataframe(train_labels,
                         directory = os.path.join(WORK_DIR, train_folder),
                         subset = "training",
                         x_col = img_col,
                         y_col = label_col,
                         color_mode='grayscale',
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "raw")


validation_datagen = ImageDataGenerator(validation_split = 0.2)

validation_generator = validation_datagen.flow_from_dataframe(train_labels,
                         directory = os.path.join(WORK_DIR, train_folder),
                         subset = "validation",
                         color_mode='grayscale',
                         x_col = img_col,
                         y_col = label_col,
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "raw")

In [ ]:
def plot_from_generator(train_generator, label_names, n=5):
    for _ in range(n):
        batch_num = randint(0, len(train_generator))
        bach_element = randint(0, BATCH_SIZE-1)

        batch = train_generator[batch_num]
        aug_images = [i/ 255 for i in batch[0]]
        aug_labels = list(batch[1])

        fig, axes = plt.subplots(1, 5, figsize = (20, 10))
        axes = axes.flatten()
        for img, lbl, ax in zip(aug_images, aug_labels, axes):
            img = np.squeeze(img)
            ax.imshow(img)
            ax.set_title(lbl)
            ax.axis('off')
        plt.tight_layout()
        plt.show()

In [ ]:
plot_from_generator(train_generator, label_names, n=5)

# Modeling

In [ ]:
def create_model():
    conv_base = EfficientNetB0(include_top = False, weights = None,
                               input_shape = (TARGET_SIZE, TARGET_SIZE, 1))
    model = conv_base.output
    model = layers.GlobalAveragePooling2D()(model)
    model = layers.Dense(len(label_names)+1, activation = "softmax")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.001),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
print('Our EfficientNet CNN has %d layers' %len(model.layers))

### Loading weights

In [ ]:
# model.load_weights('')

### Training

In [ ]:
model_name = './EffNetB0_512_8_best_weights.h5'
model_save = ModelCheckpoint(model_name, 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)


history = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save, early_stop, reduce_lr]
)

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.set_style("white")
plt.suptitle('Train history', size = 15)

ax1.plot(epochs, acc, "k--", label = "Training acc")
ax1.plot(epochs, val_acc, "ko-", label = "Validation acc")
ax1.set_title("Training and validation acc")
ax1.legend()

ax2.plot(epochs, loss, "b--", label = "Training loss", color = 'red')
ax2.plot(epochs, val_loss, "bo-", label = "Validation loss", color = 'red')
ax2.set_title("Training and validation loss")
ax2.legend()

plt.show()

In [ ]:
model.load_weights(model_name)

# Visualization of CNN intermediate activations

In [ ]:
def activation_layer_vis(img, activation_layer = 0, layers = 10):
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    fig, axes = plt.subplots(1, 1, figsize = (5, 5))
    axes.matshow(np.squeeze(activations[activation_layer][0, :, :, :]), cmap = 'viridis')
    axes.axis('off')
    plt.tight_layout()
    plt.show()

### Visualization of the first layer

In [ ]:
img_tensor = validation_generator[0][0]

In [ ]:
activation_layer_vis(img_tensor)

In [ ]:
def all_activations_vis(img, layers = 10):
    layer_outputs = [layer.output for layer in model.layers[:layers]]
    activation_model = models.Model(inputs = model.input, outputs = layer_outputs)
    activations = activation_model.predict(img)
    
    layer_names = []
    for layer in model.layers[:layers]: 
        layer_names.append(layer.name) 

    
    images_per_row = 5
    rows = np.ceil(13 / 5)
    
    plt.figure(figsize=(images_per_row*5, rows*5))
    i = 0
    for (layer_name, layer_activation) in zip(layer_names, activations): 
        n_features = layer_activation.shape[-1]

        size = layer_activation.shape[1]
        channel_image = np.squeeze(layer_activation[0, :, :, 0]) 
        channel_image -= channel_image.mean() 
        channel_image /= channel_image.std() 
        channel_image *= 64 
        channel_image += 128 
        channel_image = np.clip(channel_image, 0, 255).astype('uint8')
        
        plt.subplot(rows, images_per_row, i + 1)
        plt.imshow(channel_image)
        
        plt.title(layer_name) 
        plt.grid(False)
        plt.axis('off')
        
        i += 1
    plt.tight_layout(pad=.5)
    plt.show()

### Visualization of the first 5 layers

In [ ]:
all_activations_vis(img_tensor, 10)

Visualization of intermediate activations gives a rough step-by-step understanding of how CNN works.

# Prediction

In [ ]:
for lbl in label_names[:3]:
    img_plot(ss)

In [ ]:
preds = []
for image_id in ss[img_col]:
    image = Image.open(os.path.join(WORK_DIR, test_folder, image_id)).convert('L')
    image = image.resize((TARGET_SIZE, TARGET_SIZE))
    image = np.expand_dims(image, axis = 0)
    p = model.predict(image)
    pmax = np.argmax(p)
    preds.append(inv_label_map[pmax])
    
ss[label_col] = preds
ss[img_col] = testdex
ss

In [ ]:
ss[label_col] = preds
ss[img_col] = testdex
ss

In [ ]:
ss.to_csv('submission.csv', index = False)
print(ss.shape)

In [ ]:
!head submission.csv

In [ ]:
print("Script Complete - Runtime: {:.2f} Minutes".format((time.time() - notebookstart) / 60))